In [1]:
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import glob

import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

from tqdm.notebook import tqdm
import pandas as pd
import torch
import torch.nn as nn
import random
import os


from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

import torch.nn.functional as F
from sklearn.metrics import f1_score, accuracy_score, classification_report, recall_score, precision_score
from torch import optim



# Make DataFram

###  - columns: review_img_path, product_img_path, Normalized x,y,z ,Ppresenc

In [2]:
df = pd.read_csv('./img_Pose.csv')
df

,img_path,label,landmark_x,landmark_y,landmark_z,landmark_presence
0,./data/review_img\0_review_img\O\0.jpg,0,NaN,NaN,NaN,NaN
1,./data/review_img\0_review_img\O\1.jpg,0,"[0.7430019378662109, 0.30140167474746704, 0.80...","[0.05356726050376892, 0.04893580079078674, 0.2...","[-0.041502147912979126, 0.029880573973059654, ...","[0.9962781071662903, 0.9993698000907898, 0.990..."
2,./data/review_img\0_review_img\O\10.jpg,0,NaN,NaN,NaN,NaN
3,./data/review_img\0_review_img\O\100.jpg,0,NaN,NaN,NaN,NaN
4,./data/review_img\0_review_img\O\101.jpg,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
46417,./data/review_img\9_review_img\X\5.jpg,1,NaN,NaN,NaN,NaN
46418,./data/review_img\9_review_img\X\6.jpg,1,"[0.5282172560691833, 0.3690560460090637, 0.617...","[0.17353519797325134, 0.23485377430915833, 0.2...","[-0.15574988722801208, -0.15539097785949707, -...","[0.9999688863754272, 0.9998100399971008, 0.999..."
46419,./data/review_img\9_review_img\X\7.jpg,1,"[0.5049400329589844, 0.43606045842170715, 0.62...","[0.21291744709014893, 0.23801586031913757, 0.2...","[-0.3643159866333008, 0.3272627294063568, -0.5...","[0.9999638795852661, 0.9995668530464172, 0.999..."
46420,./data/review_img\9_review_img\X\8.jpg,1,"[0.6021363735198975, 0.43856194615364075, 0.59...","[0.08433413505554199, 0.045614272356033325, 0....","[-0.006123891565948725, -0.33828288316726685, ...","[0.9998842477798462, 0.9997095465660095, 0.999..."


In [3]:
a = df['landmark_presence'][1][1:-1]
list = [float(i.strip()) for i in df['landmark_presence'][1][1:-1].split(",")]


In [4]:
df['label'].value_counts()

label
0    42526
1     3896
Name: count, dtype: int64

In [5]:
df = df.dropna()
df = df.reset_index(drop=True)

In [6]:
df['label'].value_counts()

label
0    35675
1     2287
Name: count, dtype: int64

In [7]:
# label = []
# proper_review = glob.glob('./data/review_img/*/O/*')
# proper_review = sorted(proper_review)

# for _ in range(len(proper_review)):
#     label.append(0)
    
# unfit_review = glob.glob('./data/review_img/*/X/*')
# unfit_review = sorted(unfit_review)

# for _ in range(len(unfit_review)):
#     label.append(1)

# image_path = proper_review + unfit_review 

In [8]:
# df = pd.DataFrame(columns=['img_path','label'])
# df['img_path'] = image_path
# df['label'] = label

In [9]:
# df

# Pose Estimation Net

In [10]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [11]:
CFG = {
    'EPOCHS':100,
    'LEARNING_RATE':3e-8,
    # 'LEARNING_RATE':10,
    'BATCH_SIZE':8,
    'SEED':41
}

In [12]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = 'a'
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

#### Call Mediapipe model 

In [13]:
# base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')

# options = vision.PoseLandmarkerOptions(
#     base_options=base_options,
#     num_poses = 22,
#     output_segmentation_masks=False)

# detector = vision.PoseLandmarker.create_from_options(options)


In [14]:
# save_x = []
# save_y = []
# save_z = []
# save_presence = []

# for i in tqdm(image_path):
#     img = mp.Image.create_from_file(i)
#     pose_landmarks_list = detector.detect(img).pose_landmarks
    
#     if not pose_landmarks_list:
#         save_x.append(None)
#         save_y.append(None)
#         save_z.append(None)
#         save_presence.append(None)
#         continue
        
#     save_x.append([i.x for i in pose_landmarks_list[0][11:33]])
#     save_y.append([i.y for i in pose_landmarks_list[0][11:33]])
#     save_z.append([i.z for i in pose_landmarks_list[0][11:33]])
#     save_presence.append([i.presence for i in pose_landmarks_list[0][11:33]])
    
# df['landmark_x'] = save_x
# df['landmark_y'] = save_y
# df['landmark_z'] = save_z
# df['landmark_presence'] = save_presence

# df.to_csv('./img_Pose', index=False)
    

In [15]:
# df.to_csv('./img_Pose.csv', index=False)

In [16]:
class CustomDataset(Dataset):
    def __init__(self, img_path,landmark_x, landmark_y,	landmark_z,	landmark_presence, label):
        self.img_path = img_path

        self.landmark_x = landmark_x
        self.landmark_y = landmark_y
        self.landmark_z= landmark_z
        self.landmark_presence = landmark_presence
        
        self.label = label
        
    def __getitem__(self,index):
        
        landmarkX = [float(i.strip()) for i in self.landmark_x[index][1:-1].split(",")]
        landmarkY = [float(i.strip()) for i in self.landmark_y[index][1:-1].split(",")]
        landmarkZ = [float(i.strip()) for i in self.landmark_z[index][1:-1].split(",")]
        landmarkPre = [float(i.strip()) for i in self.landmark_presence[index][1:-1].split(",")]
        
        result = np.array(landmarkX + landmarkY + landmarkZ + landmarkPre)
        # print(result.shape)
        # print(type(result[0]))
        return result, self.label[index]
    
    def __len__(self):
        return len(self.img_path)

In [17]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

In [18]:
train_dataset = CustomDataset(train["img_path"].values, train["landmark_x"].values, train["landmark_y"].values, train["landmark_z"].values,train["landmark_presence"].values, train["label"].values)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val["img_path"].values, val["landmark_x"].values, val["landmark_y"].values, val["landmark_z"].values,val["landmark_presence"].values, val["label"].values)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [19]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.classifier1 = nn.Linear(88, 20)
        # self.classifier1 = nn.Linear(22, 2)
        self.ReLU = nn.ReLU(inplace=True)
        self.classifier2 = nn.Linear(20, 2)
        
        
    def forward(self, x):
        x = self.classifier1(x)
        x = self.ReLU(x)
        x = self.classifier2(x)
        
        return F.log_softmax(x, dim=1)
    

In [20]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []
  
    
    with torch.no_grad():
        for landmark_list, labels in tqdm(iter(val_loader)):
            
            landmark_list = landmark_list.float().to(device)
            
            labels = labels.type(torch.LongTensor).to(device)
            
            pred = model(landmark_list)

            loss = criterion(pred, labels)
            
            preds += pred.detach().argmax(1).cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
            # print(preds, true_labels)
            # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
            
        _val_loss = np.mean(val_loss)
        _val_score_f1 = f1_score(true_labels, preds, average='weighted')
        _val_score_acc = accuracy_score(true_labels, preds)
        _val_score_recall = recall_score(true_labels, preds, average='macro')
        _val_score_precision = precision_score(true_labels, preds, average='macro')
        report = classification_report(true_labels, preds)
        
        # print(accuracy_score(true_labels, preds))
        
        
    
    return _val_loss, _val_score_f1, _val_score_acc, report , _val_score_recall, _val_score_precision

In [21]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model = model.to(device)
    criterion = nn.NLLLoss(weight=torch.tensor([0.01, 0.99]), reduction="sum").to(device)

    best_score = 0
    best_model = None
    
    for epoch in range(0, CFG['EPOCHS']):
        model.train()
        train_loss = []
        
        for landmark_list, labels in tqdm(iter(train_loader)):
            
            landmark_list = landmark_list.float().to(device)
            
            labels = labels.type(torch.LongTensor).to(device)
            
            optimizer.zero_grad()
            output = model(landmark_list)
            
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
                    
                
        _val_loss, _val_score_f1, _val_score_acc, report , _val_score_recall, _val_score_precision = validation(model, criterion, val_loader, device)
        
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val accuracy score : [{_val_score_acc:.5f}] Val recall score : [{_val_score_recall:.5f}] Val f1 score : [{_val_score_f1:.5f}]')
        print(report)
        if scheduler is not None:
            scheduler.step(_val_score_recall)
            
        if best_score > _val_score_recall:
            best_score = _val_score_recall
            best_model = model
            torch.save(best_model.state_dict("./Pose_Estimate.pt"))
            
            
    
    return best_model

In [22]:
model = BaseModel()
model.eval()

BaseModel(
  (classifier1): Linear(in_features=88, out_features=20, bias=True)
  (ReLU): ReLU(inplace=True)
  (classifier2): Linear(in_features=20, out_features=2, bias=True)
)

In [23]:
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/3322 [00:00<?, ?it/s]

  0%|          | 0/1424 [00:00<?, ?it/s]

Epoch [0], Train Loss : [0.43047] Val Loss : [0.42799] Val accuracy score : [0.93959] Val precision score : [0.68440] Val f1 score : [0.91151]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97     10703
           1       0.43      0.01      0.02       686

    accuracy                           0.94     11389
   macro avg       0.68      0.50      0.49     11389
weighted avg       0.91      0.94      0.91     11389



  0%|          | 0/3322 [00:00<?, ?it/s]

  0%|          | 0/1424 [00:00<?, ?it/s]

...

  0%|          | 0/3322 [00:00<?, ?it/s]

  0%|          | 0/1424 [00:00<?, ?it/s]

Epoch [15], Train Loss : [0.41874] Val Loss : [0.41679] Val accuracy score : [0.93889] Val precision score : [0.63691] Val f1 score : [0.91180]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97     10703
           1       0.33      0.01      0.03       686

    accuracy                           0.94     11389
   macro avg       0.64      0.51      0.50     11389
weighted avg       0.90      0.94      0.91     11389



  0%|          | 0/3322 [00:00<?, ?it/s]

KeyboardInterrupt: 